In [18]:
# 데이터 처리 
import pandas as pd 
import numpy as np
# 시각화 
import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt 
# 시각화 옵션
from IPython.display import set_matplotlib_formats
# 한글 글꼴 설정
mpl.rc('font',family ='Malgun Gothic')
# 음수값 설정 
mpl.rc('axes',unicode_minus= True )
# 그래프 선명하게 출력
set_matplotlib_formats('retina')


import pandas as pd
import numpy as np
import re

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential

In [2]:
train      = pd.read_csv("train_data.csv")
test       = pd.read_csv("test_data.csv")
submission = pd.read_csv("sample_submission.csv")
topic_dict = pd.read_csv("topic_dict.csv")

In [3]:
df1 = pd.read_csv('train_data.csv')
print(df1.shape)
df1.head()

(45654, 3)


,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [4]:
X_train = df1['title']
Y_train = df1['topic_idx']

- 불용어 및 특수문자 처리

In [5]:
from konlpy.tag import Okt
import re 

In [6]:
df_stopword  = pd.read_csv('stopword.txt',header=None, names=['불용어'])
stopword_set = set(df_stopword['불용어'].values.tolist())

In [7]:
# 불용어 및 특수 기호 처리함수 구성 
def stopword_function(X): # Series 형태로 Input 
    clean_train_review = []
    okt = Okt()
    # 모든 X 값에 대해 불용어 및 특수기호 처리 
    for i in X:

        # 문자데이터에 대한 불용어 처리 
        if type(i) == str:
            # 특수 문자 처리 
            review_text = re.sub("[!1(+.),.@@\/n2345']?^가-힣ㄱ-ㅎㅏ-ㅣ\\s]","",i) 
            # Tokenizing & Stemming 
            word_text = okt.morphs(review_text,stem=True)
            # StopWord 
            word_text = [token for token in word_text if not token in stopword_set]  

            clean_train_review.append(word_text)

        # 숫자데이터가 오는 경우, 공백을 append     
        else:
            clean_train_review.append([])
            
    return clean_train_review

In [8]:
X_train_clean = stopword_function(X_train) 불용어 처리

In [9]:
X_length = X_train.apply(len)
X_length.describe()

# max 44 으로 잡는걸로 결정

count    45654.000000
mean        27.330552
std          4.968172
min          4.000000
25%         25.000000
50%         28.000000
75%         31.000000
max         44.000000
Name: title, dtype: float64

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
model_token = Tokenizer()
model_token.fit_on_texts(X_train_clean)

In [14]:
# Embedding Layer 구성 작업, Input Unit  
len(model_token.word_index)

30505

In [19]:
X_sequence = model_token.texts_to_sequences(X_train_clean)

max_length = 44 
train_x    = pad_sequences(X_sequence, maxlen=max_length, padding='post')

In [21]:
train_x.max()

30505

In [22]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras        import layers
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.utils  import to_categorical

# Random Seed 

tensorflow.random.set_seed(7)

In [23]:
# Y 값을 Matrix 형태로 변환
train_y = to_categorical(Y_train.astype(int))
train_y

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

- Test Set 데이터 분류 실시 

In [24]:
# 데이터 불러오기 
df_test = pd.read_csv("test_data.csv")
print(df_test.shape)
df_test.head(2)

(9131, 2)


,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사


In [25]:
# 특수문자처리 / 불용어 처리 
X_test_clean = stopword_function(df_test['title'])

In [26]:
# Text to Sequence / Padding 
X_test_sequence = model_token.texts_to_sequences(X_test_clean)
test_x    = pad_sequences(X_test_sequence,maxlen=max_length, padding='post')

In [123]:
# 전처리 완료

- Model

In [124]:
print(train_x.shape, test_x.shape)

(45654, 45) (9131, 45)


In [27]:
es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)

In [37]:
seed=7

model_lstm = Sequential()

model_lstm.add(layers.Embedding(31000,64, input_length=max_length))

model_lstm.add(Dropout(0.2)) # ADDED

model_lstm.add(layers.Bidirectional(layers.LSTM(32)))

model_lstm.add(Dropout(0.2)) # ADDED





model_lstm.add(layers.Dense(7, activation='softmax'))

model_lstm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['acc'])

model_lstm.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 44, 64)            1984000   
_________________________________________________________________
dropout_8 (Dropout)          (None, 44, 64)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                24832     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 455       
Total params: 2,009,287
Trainable params: 2,009,287
Non-trainable params: 0
_________________________________________________________________


In [35]:
history = model_lstm.fit(train_x, train_y,
                        epochs=8, batch_size=180, validation_split=0.2, callbacks=es)

Epoch 1/8
203/203 [==============================] - 7s 20ms/step - loss: 1.3170 - acc: 0.5707 - val_loss: 1.1957 - val_acc: 0.5889
Epoch 2/8
203/203 [==============================] - 3s 17ms/step - loss: 0.6090 - acc: 0.8128 - val_loss: 0.7913 - val_acc: 0.7412
Epoch 3/8
203/203 [==============================] - 3s 16ms/step - loss: 0.4270 - acc: 0.8695 - val_loss: 0.7470 - val_acc: 0.7466
Epoch 4/8
203/203 [==============================] - 3s 16ms/step - loss: 0.3479 - acc: 0.8925 - val_loss: 0.6877 - val_acc: 0.7665
Epoch 5/8
203/203 [==============================] - 3s 16ms/step - loss: 0.2981 - acc: 0.9083 - val_loss: 0.6192 - val_acc: 0.7892
Epoch 6/8
203/203 [==============================] - 3s 17ms/step - loss: 0.2612 - acc: 0.9187 - val_loss: 0.6301 - val_acc: 0.7863
Epoch 7/8
203/203 [==============================] - 3s 16ms/step - loss: 0.2329 - acc: 0.9273 - val_loss: 0.6264 - val_acc: 0.7910
Epoch 8/8
203/203 [==============================] - 3s 16ms/step - loss: 0.

In [38]:
# 계층 교차 검증
n_fold = 10 
seed = 21

cv = StratifiedKFold(n_splits = n_fold, shuffle=True, random_state=seed)

# 테스트데이터의 예측값 담을 곳 생성
test_y = np.zeros((test_x.shape[0], 7))

# 조기 종료 옵션 추가

es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)

for i, (i_trn, i_val) in enumerate(cv.split(train_x, Y_train), 1):
    print(f'training model for CV #{i}')

    model_lstm.fit(train_x[i_trn], 
            to_categorical(Y_train[i_trn]),
            validation_data=(train_x[i_val], to_categorical(Y_train[i_val])),
            epochs=20,
            batch_size=180,
            callbacks=[es])     # 조기 종료 옵션
                      
    test_y += model_lstm.predict(test_x) / n_fold    # 나온 예측값들을 교차 검증 횟수로 나눈다

training model for CV #1
Epoch 1/20
229/229 [==============================] - 6s 18ms/step - loss: 1.3829 - acc: 0.5169 - val_loss: 0.8298 - val_acc: 0.7608
Epoch 2/20
229/229 [==============================] - 4s 16ms/step - loss: 0.5914 - acc: 0.8164 - val_loss: 0.5283 - val_acc: 0.8254
Epoch 3/20
229/229 [==============================] - 4s 16ms/step - loss: 0.4052 - acc: 0.8752 - val_loss: 0.4782 - val_acc: 0.8408
Epoch 4/20
229/229 [==============================] - 4s 16ms/step - loss: 0.3305 - acc: 0.8965 - val_loss: 0.4592 - val_acc: 0.8537
Epoch 5/20
229/229 [==============================] - 4s 16ms/step - loss: 0.2833 - acc: 0.9107 - val_loss: 0.4883 - val_acc: 0.8463
Epoch 6/20
229/229 [==============================] - 4s 16ms/step - loss: 0.2516 - acc: 0.9202 - val_loss: 0.5041 - val_acc: 0.8469
Epoch 7/20
229/229 [==============================] - 4s 16ms/step - loss: 0.2252 - acc: 0.9284 - val_loss: 0.5156 - val_acc: 0.8445
Restoring model weights from the end of the 

- Test Data

In [39]:
# 테스트 데이터의 예측값 확인
test_y

array([[4.53738671e-02, 2.68199232e-02, 1.10607434e-01, ...,
        4.88410364e-03, 1.45945586e-04, 1.00344975e-03],
       [2.57554324e-04, 2.10724219e-04, 7.49938201e-04, ...,
        4.54464353e-04, 3.44279864e-06, 1.10758785e-04],
       [7.92950790e-03, 2.73435775e-02, 9.08840664e-01, ...,
        1.30608545e-03, 1.06155214e-03, 4.02068116e-02],
       ...,
       [3.55404766e-02, 3.10791957e-03, 2.95060650e-01, ...,
        4.17715490e-02, 2.77512043e-04, 1.51880182e-03],
       [2.04279698e-01, 2.74182970e-02, 7.61268716e-01, ...,
        1.73126692e-03, 9.69653881e-05, 1.27290674e-03],
       [3.80025419e-03, 2.98764165e-03, 6.35839196e-01, ...,
        8.96834365e-03, 8.62448170e-04, 3.12454928e-01]])

In [40]:
topic = []
for i in range(len(test_y)):
    topic.append(np.argmax(test_y[i]))

In [133]:
# sample_submission 에 대입
submission['topic_idx'] = topic
submission

,index,topic_idx
0,45654,3
1,45655,3
2,45656,2
3,45657,0
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,3
9129,54783,2


In [42]:
submission.to_csv('자연어_cv02.csv',index = False)

In [43]:
tmp_pred = model_lstm.predict(test_x)

pred = np.argmax(tmp_pred, axis = 1)

In [44]:
submission.topic_idx = pred
submission.sample(10)

,index,topic_idx
8924,54578,2
1079,46733,4
8298,53952,2
4829,50483,3
1183,46837,0
5935,51589,0
4645,50299,6
6989,52643,5
4968,50622,5
4059,49713,6
